<a href="https://colab.research.google.com/github/cufuentes/CodingDojo/blob/main/pipelines_actividades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')

In [4]:
path = "/content/Cereal with missing values.xlsx"
df = pd.read_excel (path)
df.head()

,name,Manufacturer,type,calories per serving,grams of protein,grams of fat,milligrams of sodium,grams of dietary fiber,grams of complex carbohydrates,grams of sugars,milligrams of potassium,vitamins and minerals (% of FDA recommendation),Display shelf,Weight in ounces per one serving,Number of cups in one serving,Rating of cereal
0,Apple Cinnamon Cheerios,General Mills,Cold,110.0,2,2.0,180.0,1.5,10.5,10.0,70,25.0,1,1.00,0.75,29.509541
1,Basic 4,General Mills,Cold,130.0,3,2.0,NaN,2.0,18.0,NaN,100,25.0,3,1.33,0.75,37.038562
2,Cheerios,General Mills,Cold,NaN,6,2.0,290.0,2.0,17.0,1.0,105,25.0,1,1.00,1.25,50.764999
3,Cinnamon Toast Crunch,General Mills,Cold,120.0,1,3.0,210.0,0.0,13.0,9.0,45,25.0,2,1.00,0.75,19.823573
4,Clusters,General Mills,Cold,110.0,3,2.0,140.0,2.0,13.0,7.0,105,25.0,3,1.00,0.50,40.400208


In [9]:
# spliteamos en X e Y , train y test
y = df ["calories per serving"]
X = df.drop(columns = "calories per serving",axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state =42)

In [10]:
#revisamos columnas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 16 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   name                                             77 non-null     object 
 1   Manufacturer                                     77 non-null     object 
 2   type                                             68 non-null     object 
 3   calories per serving                             70 non-null     float64
 4   grams of protein                                 77 non-null     int64  
 5   grams of fat                                     69 non-null     float64
 6   milligrams of sodium                             76 non-null     float64
 7   grams of dietary fiber                           77 non-null     float64
 8   grams of complex carbohydrates                   77 non-null     float64
 9   grams of sugars                   

#   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0  -  name                                             nominal     object
 1   Manufacturer                                     nominal     object
 2   type                                             nominal     object
 3   calories per serving                             numerica     float64
 4   grams of protein                                 numerica     int64  
 5   grams of fat                                     numerica     float64
 6   milligrams of sodium                             numerica     float64
 7   grams of dietary fiber                           numerica     float64
 8   grams of complex carbohydrates                   numerica     float64
 9   grams of sugars                                  numerica     float64
 10  milligrams of potassium                          numerica     int64  
 11  vitamins and minerals (% of FDA recommendation)  numerica     float64
 12  Display shelf                                    numerica     int64  
 13  Weight in ounces per one serving                 numerica     float64
 14  Number of cups in one serving                    numerica     float64
 15  Rating of cereal                                 numerica     float64



In [17]:
#seguimos inspeccionando datos
print(df["name"].value_counts())

name
Apple Cinnamon Cheerios      1
Strawberry Fruit Wheats      1
Great Grains Pecan           1
Grape-Nuts                   1
Grape Nuts Flakes            1
                            ..
Corn Flakes                  1
Apple Jacks                  1
All-Bran with Extra Fiber    1
All-Bran                     1
Quaker Oatmeal               1
Name: count, Length: 77, dtype: int64


In [18]:
print(df["Manufacturer"].value_counts())

Manufacturer
Kelloggs                       23
General Mills                  22
Post                            9
Quaker Oats                     8
Ralston Purina                  8
Nabisco                         6
American Home Food Products     1
Name: count, dtype: int64


In [19]:
print(df["type"].value_counts())

type
Cold    65
Hot      3
Name: count, dtype: int64


In [20]:
# definimos selectores de columnas
cat_selector = make_column_selector(dtype_include = "object")
num_selector = make_column_selector(dtype_include = "number")


In [22]:
# definimos imputers

mean_imputer = SimpleImputer(strategy = "mean")
freq_imputer = SimpleImputer(strategy = "most_frequent")

# One Hot Encoder

ohe = OneHotEncoder(handle_unknown="ignore",sparse=False)

# Scaler

scaler = StandardScaler()

In [23]:
# definimos pipeline numerico

num_pipeline = make_pipeline (mean_imputer , scaler)
num_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [24]:
#definimos pipeline categorico

cat_pipeline = make_pipeline (freq_imputer , ohe)
cat_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [28]:
#tuplas para column transformer
number_tuple = (num_pipeline , num_selector)
cat_tuple= (cat_pipeline, cat_selector)

# column transformer

preprocessor = make_column_transformer (number_tuple , cat_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7de39484ae60>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7de394848ca0>)])

In [29]:
# fit on train
preprocessor.fit(X_train)



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7de39484ae60>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7de394848ca0>)])

In [30]:
#transformar train y test
X_train_pro = preprocessor.transform(X_train)
X_test_pro = preprocessor.transform(X_test)

In [31]:
X_train_pro

array([[-1.30301442, -0.97467943,  0.56162348, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.40438378,  0.        ,  0.68120871, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.40438378, -0.97467943,  1.99664622, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 1.25808288,  1.94935887, -0.03630266, ...,  1.        ,
         1.        ,  0.        ],
       [ 0.40438378,  0.97467943, -0.15588789, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.40438378,  0.        ,  0.08328257, ...,  0.        ,
         1.        ,  0.        ]])

In [32]:
X_test_pro

array([[ 0.40438378,  0.97467943, -0.15588789, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.40438378,  0.97467943, -0.03630266, ...,  0.        ,
         1.        ,  0.        ],
       [-0.44931532,  0.        ,  1.27913485, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-1.30301442,  0.        ,  0.32245303, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.40438378,  0.        ,  0.2028678 , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.25808288,  0.        ,  1.27913485, ...,  0.        ,
         1.        ,  0.        ]])